In [1]:
import pandas as pd
import numpy as np
import time
import json
import os
#import off_label
#import vocab_mappings as vocab

Goal is to extract all off label indications to be able to map them to higher level categories for representation

Loading DC Summary Results

Once this is figured out - need to go back to agg results file and adjust so that I have the non heirarchical results as well 

In [2]:
DC_summary_final = pd.read_csv('/Users/khs2138/OneDrive - cumc.columbia.edu/Archive/Classes/Symbolic_Methods/Symbolic_Project/DC_summary_Final.csv')
                               

In [3]:
DC_summary_final[['Off-Label']].head()

,Off-Label
0,"[436096, 442077, 40405599, 4169287]"
1,"[436096, 436962, 442077, 40405599]"
2,"[436096, 436962, 40405599, 31967]"
3,"[436096, 43530621, 40405599]"
4,"[436096, 4218106, 442077]"


Formatting results - splitting out snomed codes by commas

In [4]:
df = pd.Series(DC_summary_final['Off-Label'].tolist()).str.split(',', expand=True)
df.head()

,0,1,2,3
0,[436096,442077,40405599,4169287]
1,[436096,436962,442077,40405599]
2,[436096,436962,40405599,31967]
3,[436096,43530621,40405599],None
4,[436096,4218106,442077],None


Removing brackets

In [5]:
df = df[[0,1,2,3]].replace({'\\[':''}, regex=True)
df = df[[0,1,2,3]].replace({'\\]':''}, regex=True)
df = df[[0,1,2,3]].replace({' ':''}, regex=True)
df.head()

,0,1,2,3
0,436096,442077,40405599,4169287
1,436096,436962,442077,40405599
2,436096,436962,40405599,31967
3,436096,43530621,40405599,None
4,436096,4218106,442077,None


In [6]:
df.values.tolist()

[['436096', '442077', '40405599', '4169287'],
 ['436096', '436962', '442077', '40405599'],
 ['436096', '436962', '40405599', '31967'],
 ['436096', '43530621', '40405599', None],
 ['436096', '4218106', '442077', None],
 ['436096', '442077', '40405599', None],
 ['436096', '40405599', None, None],
 ['436096', '43530621', '40405599', None],
 ['436096', '442077', '31967', None],
 ['4218106', '31967', None, None],
 ['436096', '40405599', None, None],
 ['4218106', '40405599', None, None],
 ['436096', '40405599', None, None],
 ['436096', '40405599', None, None],
 ['43530621', '40405599', None, None],
 ['436096', '40405599', None, None],
 ['4218106', '31967', None, None],
 ['436096', '40405599', None, None],
 ['436096', '40405599', None, None],
 ['4218106', '31967', None, None],
 ['436096', '40405599', None, None],
 ['436096', '40405599', None, None],
 ['436096', '40405599', None, None],
 ['436096', '40405599', None, None],
 ['4195384', '442077', None, None],
 ['436096', '40405599', None, None]

In [7]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [8]:
test_list = flatten(df.values.tolist())

In [9]:
test_list = [i for i in test_list if i is not None]


In [10]:
set(test_list)

{'',
 '31967',
 '40405599',
 '4169287',
 '4195384',
 '4218106',
 '43530621',
 '43531003',
 '436096',
 '436962',
 '442077'}

In [11]:
df_agg = pd.DataFrame(pd.DataFrame(test_list)[0].value_counts()).reset_index().rename(columns={'index':'SNOMED',0:'Count'})
df_agg = df_agg.drop(0).reset_index(drop=True)
df_agg['SNOMED'] = df_agg['SNOMED'].astype(int)
df_agg


,SNOMED,Count
0,40405599,21
1,436096,21
2,442077,13
3,4218106,7
4,436962,6
5,31967,6
6,43530621,5
7,4195384,1
8,43531003,1
9,4169287,1


Compare back to OMOP tables


In [12]:
# 1. LOAD OMOP TABLES

# Change directory to where OMOP Vocabulary download is saved
os.chdir('/Users/khs2138/OneDrive - cumc.columbia.edu/Archive/Classes/Symbolic_Methods/Symbolic_Project/OMOP_vocabulary')
# Load all concepts
concept = pd.read_csv('CONCEPT.csv',delimiter='\t',skiprows=1,
                      names = ['concept_id','concept_name',
                              'domain_id','vocabulary_id','concept_class_id','standard_concept',
                              'concept_code','valid_start_date','valid_end_date','invalid_reason'],index_col=False)
# Load all concept relationships
concept_relationship = pd.read_csv('CONCEPT_RELATIONSHIP.csv',delimiter='\t',skiprows=1,
                      names = ['concept_id_1','concept_id_2',
                              'relationship_id',
                               'valid_start_date','valid_end_date','invalid_reason'],
                           index_col=False)

/Users/khs2138/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5,6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [13]:
df_agg_concept = pd.merge(df_agg, concept[['concept_id','concept_name','domain_id','vocabulary_id','concept_class_id','standard_concept']], how="left", left_on='SNOMED', right_on='concept_id')

In [14]:
df_agg_concept

,SNOMED,Count,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept
0,40405599,21,40405599,Fibromyalgia,Condition,SNOMED,Clinical Finding,S
1,436096,21,436096,Chronic pain,Condition,SNOMED,Clinical Finding,S
2,442077,13,442077,Anxiety disorder,Condition,SNOMED,Clinical Finding,S
3,4218106,7,4218106,Alcoholism,Condition,SNOMED,Clinical Finding,S
4,436962,6,436962,Insomnia,Condition,SNOMED,Clinical Finding,S
5,31967,6,31967,Nausea,Condition,SNOMED,Clinical Finding,S
6,43530621,5,43530621,Acute postoperative pain,Condition,SNOMED,Clinical Finding,S
7,4195384,1,4195384,Chronic cough,Condition,SNOMED,Clinical Finding,S
8,43531003,1,43531003,Essential tremor,Condition,SNOMED,Clinical Finding,S
9,4169287,1,4169287,Itching of skin,Condition,SNOMED,Clinical Finding,S


Getting first degree parents

In [15]:
parents = []
for i,j in df_agg.iterrows():
    SNOMED = df_agg['SNOMED'][i]
    parents_temp = concept_relationship.loc[(concept_relationship['concept_id_1']==SNOMED)&
                                                (concept_relationship['relationship_id']=='Is a') &
                                                    (concept_relationship['concept_id_2']!='Clinical Finding')]['concept_id_2'].values.tolist() 
    
    parents.append(parents_temp)
print(parents)

[[4160062, 4132926, 137275, 436096, 443783], [4118269, 4329041], [432586, 441542], [36714559, 4080762], [435657, 439708], [4304916], [433456, 439502], [254761], [372604], [4090425, 4302654]]


In [17]:
df_agg_concept['parents_1'] = pd.Series(parents)

In [18]:
df_agg_concept

,SNOMED,Count,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,parents_1
0,40405599,21,40405599,Fibromyalgia,Condition,SNOMED,Clinical Finding,S,"[4160062, 4132926, 137275, 436096, 443783]"
1,436096,21,436096,Chronic pain,Condition,SNOMED,Clinical Finding,S,"[4118269, 4329041]"
2,442077,13,442077,Anxiety disorder,Condition,SNOMED,Clinical Finding,S,"[432586, 441542]"
3,4218106,7,4218106,Alcoholism,Condition,SNOMED,Clinical Finding,S,"[36714559, 4080762]"
4,436962,6,436962,Insomnia,Condition,SNOMED,Clinical Finding,S,"[435657, 439708]"
5,31967,6,31967,Nausea,Condition,SNOMED,Clinical Finding,S,[4304916]
6,43530621,5,43530621,Acute postoperative pain,Condition,SNOMED,Clinical Finding,S,"[433456, 439502]"
7,4195384,1,4195384,Chronic cough,Condition,SNOMED,Clinical Finding,S,[254761]
8,43531003,1,43531003,Essential tremor,Condition,SNOMED,Clinical Finding,S,[372604]
9,4169287,1,4169287,Itching of skin,Condition,SNOMED,Clinical Finding,S,"[4090425, 4302654]"


Getting second degree parents

In [38]:
parents_all = []
for i,j in df_agg_concept.iterrows():
    parents_row = []
    for k in df_agg_concept['parents_1'][i]:
        SNOMED = k
        parents_temp = concept_relationship.loc[(concept_relationship['concept_id_1']==SNOMED)&
                                                (concept_relationship['relationship_id']=='Is a') &
                                                    (concept_relationship['concept_id_2']!='Clinical Finding')]['concept_id_2'].values.tolist() 
        parents_row = parents_row + parents_temp
        print(parents_row)
    parents_all.append(list(set(parents_row)))
    

[4274025, 4329041]
[4274025, 4329041, 44783587, 4329041]
[4274025, 4329041, 44783587, 4329041, 4024566, 4047779]
[4274025, 4329041, 44783587, 4329041, 4024566, 4047779, 4118269, 4329041]
[4274025, 4329041, 44783587, 4329041, 4024566, 4047779, 4118269, 4329041, 4274025]
[4329041]
[4329041, 4170962]
[4274025]
[4274025, 4025215]
[444363]
[444363, 440069, 4004672]
[435524]
[435524, 435524]
[4302537]
[4118269, 4329041]
[4118269, 4329041, 4002014]
[4267789, 4103331]
[4041681]
[4182969]
[4182969, 4024013]


In [42]:
df_agg_concept['parents_2'] = pd.Series(parents_all)
df_agg_concept

,SNOMED,Count,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,parents_1,parents_2
0,40405599,21,40405599,Fibromyalgia,Condition,SNOMED,Clinical Finding,S,"[4160062, 4132926, 137275, 436096, 443783]","[44783587, 4047779, 4274025, 4329041, 4024566,..."
1,436096,21,436096,Chronic pain,Condition,SNOMED,Clinical Finding,S,"[4118269, 4329041]","[4329041, 4170962]"
2,442077,13,442077,Anxiety disorder,Condition,SNOMED,Clinical Finding,S,"[432586, 441542]","[4274025, 4025215]"
3,4218106,7,4218106,Alcoholism,Condition,SNOMED,Clinical Finding,S,"[36714559, 4080762]","[4004672, 444363, 440069]"
4,436962,6,436962,Insomnia,Condition,SNOMED,Clinical Finding,S,"[435657, 439708]",[435524]
5,31967,6,31967,Nausea,Condition,SNOMED,Clinical Finding,S,[4304916],[4302537]
6,43530621,5,43530621,Acute postoperative pain,Condition,SNOMED,Clinical Finding,S,"[433456, 439502]","[4329041, 4118269, 4002014]"
7,4195384,1,4195384,Chronic cough,Condition,SNOMED,Clinical Finding,S,[254761],"[4103331, 4267789]"
8,43531003,1,43531003,Essential tremor,Condition,SNOMED,Clinical Finding,S,[372604],[4041681]
9,4169287,1,4169287,Itching of skin,Condition,SNOMED,Clinical Finding,S,"[4090425, 4302654]","[4182969, 4024013]"


Third degree parents

In [43]:
parents_all = []
for i,j in df_agg_concept.iterrows():
    parents_row = []
    for k in df_agg_concept['parents_2'][i]:
        SNOMED = k
        parents_temp = concept_relationship.loc[(concept_relationship['concept_id_1']==SNOMED)&
                                                (concept_relationship['relationship_id']=='Is a') &
                                                    (concept_relationship['concept_id_2']!='Clinical Finding')]['concept_id_2'].values.tolist() 
        parents_row = parents_row + parents_temp
        print(parents_row)
    parents_all.append(list(set(parents_row)))

[4024013, 4042140]
[4024013, 4042140, 4274025, 4042140]
[4024013, 4042140, 4274025, 4042140, 441840]
[4024013, 4042140, 4274025, 4042140, 441840, 4170962]
[4024013, 4042140, 4274025, 4042140, 441840, 4170962, 4042140]
[4024013, 4042140, 4274025, 4042140, 441840, 4170962, 4042140, 4329041]
[4170962]
[4170962, 4024013]
[441840]
[441840, 4269314]
[40483111]
[40483111, 4274025]
[40483111, 4274025, 444363]
[444108, 4274025]
[4042140]
[4170962]
[4170962, 4329041]
[4170962, 4329041, 4329041]
[4024567]
[4024567, 4024567]
[376337]
[141960, 44783587]
[141960, 44783587, 4011630]


In [44]:
df_agg_concept['parents_3'] = pd.Series(parents_all)
df_agg_concept

,SNOMED,Count,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,parents_1,parents_2,parents_3
0,40405599,21,40405599,Fibromyalgia,Condition,SNOMED,Clinical Finding,S,"[4160062, 4132926, 137275, 436096, 443783]","[44783587, 4047779, 4274025, 4329041, 4024566,...","[4274025, 4024013, 441840, 4329041, 4170962, 4..."
1,436096,21,436096,Chronic pain,Condition,SNOMED,Clinical Finding,S,"[4118269, 4329041]","[4329041, 4170962]","[4170962, 4024013]"
2,442077,13,442077,Anxiety disorder,Condition,SNOMED,Clinical Finding,S,"[432586, 441542]","[4274025, 4025215]","[441840, 4269314]"
3,4218106,7,4218106,Alcoholism,Condition,SNOMED,Clinical Finding,S,"[36714559, 4080762]","[4004672, 444363, 440069]","[4274025, 444363, 40483111]"
4,436962,6,436962,Insomnia,Condition,SNOMED,Clinical Finding,S,"[435657, 439708]",[435524],"[4274025, 444108]"
5,31967,6,31967,Nausea,Condition,SNOMED,Clinical Finding,S,[4304916],[4302537],[4042140]
6,43530621,5,43530621,Acute postoperative pain,Condition,SNOMED,Clinical Finding,S,"[433456, 439502]","[4329041, 4118269, 4002014]","[4329041, 4170962]"
7,4195384,1,4195384,Chronic cough,Condition,SNOMED,Clinical Finding,S,[254761],"[4103331, 4267789]",[4024567]
8,43531003,1,43531003,Essential tremor,Condition,SNOMED,Clinical Finding,S,[372604],[4041681],[376337]
9,4169287,1,4169287,Itching of skin,Condition,SNOMED,Clinical Finding,S,"[4090425, 4302654]","[4182969, 4024013]","[141960, 44783587, 4011630]"


In [28]:
flatten(df_agg_concept['parents_1'].tolist())

[4160062,
 4132926,
 137275,
 436096,
 443783,
 4118269,
 4329041,
 432586,
 441542,
 36714559,
 4080762,
 435657,
 439708,
 4304916,
 433456,
 439502,
 254761,
 372604,
 4090425,
 4302654]

# Testing extracting the df to represent as a graph

In [104]:
df_concept = pd.merge(df_agg, concept[['concept_id','concept_name','domain_id','vocabulary_id','concept_class_id','standard_concept']], 
                    how="left", left_on='SNOMED', right_on='concept_id')
df_concept

,SNOMED,Count,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept
0,40405599,21,40405599,Fibromyalgia,Condition,SNOMED,Clinical Finding,S
1,436096,21,436096,Chronic pain,Condition,SNOMED,Clinical Finding,S
2,442077,13,442077,Anxiety disorder,Condition,SNOMED,Clinical Finding,S
3,4218106,7,4218106,Alcoholism,Condition,SNOMED,Clinical Finding,S
4,436962,6,436962,Insomnia,Condition,SNOMED,Clinical Finding,S
5,31967,6,31967,Nausea,Condition,SNOMED,Clinical Finding,S
6,43530621,5,43530621,Acute postoperative pain,Condition,SNOMED,Clinical Finding,S
7,4195384,1,4195384,Chronic cough,Condition,SNOMED,Clinical Finding,S
8,43531003,1,43531003,Essential tremor,Condition,SNOMED,Clinical Finding,S
9,4169287,1,4169287,Itching of skin,Condition,SNOMED,Clinical Finding,S


In [105]:
df_concept_rel = pd.merge(df_concept,concept_relationship[['concept_id_1','concept_id_2','relationship_id']],
                     how = 'left',left_on = 'concept_id',right_on = 'concept_id_1')

df_concept_rel = df_concept_rel.loc[df_concept_rel['relationship_id'] == 'Is a']
df_concept_rel


,SNOMED,Count,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_id_1,concept_id_2,relationship_id
2,40405599,21,40405599,Fibromyalgia,Condition,SNOMED,Clinical Finding,S,40405599,4160062,Is a
20,40405599,21,40405599,Fibromyalgia,Condition,SNOMED,Clinical Finding,S,40405599,4132926,Is a
34,40405599,21,40405599,Fibromyalgia,Condition,SNOMED,Clinical Finding,S,40405599,137275,Is a
35,40405599,21,40405599,Fibromyalgia,Condition,SNOMED,Clinical Finding,S,40405599,436096,Is a
36,40405599,21,40405599,Fibromyalgia,Condition,SNOMED,Clinical Finding,S,40405599,443783,Is a
75,436096,21,436096,Chronic pain,Condition,SNOMED,Clinical Finding,S,436096,4118269,Is a
140,436096,21,436096,Chronic pain,Condition,SNOMED,Clinical Finding,S,436096,4329041,Is a
193,442077,13,442077,Anxiety disorder,Condition,SNOMED,Clinical Finding,S,442077,432586,Is a
215,442077,13,442077,Anxiety disorder,Condition,SNOMED,Clinical Finding,S,442077,441542,Is a
247,4218106,7,4218106,Alcoholism,Condition,SNOMED,Clinical Finding,S,4218106,36714559,Is a


Second level

In [112]:
df_agg2 = pd.DataFrame(df_concept_rel['concept_id_2'].unique()).rename(columns={0:'SNOMED'})

In [115]:
df_concept2 = pd.merge(df_agg2, concept[['concept_id','concept_name','domain_id','vocabulary_id','concept_class_id','standard_concept']], 
                    how="left", left_on='SNOMED', right_on='concept_id')
df_concept2 = df_concept2.loc[df_concept2['standard_concept'] == 'S']
df_concept2

,SNOMED,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept
0,4160062,4160062,Disorder characterized by pain,Condition,SNOMED,Clinical Finding,S
1,4132926,4132926,Pain finding at anatomical site,Condition,SNOMED,Clinical Finding,S
2,137275,137275,Disorder of muscle,Condition,SNOMED,Clinical Finding,S
3,436096,436096,Chronic pain,Condition,SNOMED,Clinical Finding,S
4,443783,443783,Chronic disease,Condition,SNOMED,Clinical Finding,S
6,4329041,4329041,Pain,Condition,SNOMED,Clinical Finding,S
7,432586,432586,Mental disorder,Condition,SNOMED,Clinical Finding,S
8,441542,441542,Anxiety,Condition,SNOMED,Clinical Finding,S
9,36714559,36714559,Disorder caused by alcohol,Condition,SNOMED,Clinical Finding,S
10,4080762,4080762,Psychoactive substance dependence,Condition,SNOMED,Clinical Finding,S


In [118]:
df_concept_rel2 = pd.merge(df_concept2,concept_relationship[['concept_id_1','concept_id_2','relationship_id']],
                     how = 'left',left_on = 'concept_id',right_on = 'concept_id_1')

df_concept_rel2 = df_concept_rel2.loc[df_concept_rel2['relationship_id'] == 'Is a']
df_concept_rel2 = df_concept_rel2.loc[df_concept_rel2['concept_name'] != 'Clinical Finding']
df_concept_rel2

,SNOMED,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_id_1,concept_id_2,relationship_id
26,4160062,4160062,Disorder characterized by pain,Condition,SNOMED,Clinical Finding,S,4160062,4274025,Is a
114,4160062,4160062,Disorder characterized by pain,Condition,SNOMED,Clinical Finding,S,4160062,4329041,Is a
168,4132926,4132926,Pain finding at anatomical site,Condition,SNOMED,Clinical Finding,S,4132926,44783587,Is a
188,4132926,4132926,Pain finding at anatomical site,Condition,SNOMED,Clinical Finding,S,4132926,4329041,Is a
263,137275,137275,Disorder of muscle,Condition,SNOMED,Clinical Finding,S,137275,4024566,Is a
400,137275,137275,Disorder of muscle,Condition,SNOMED,Clinical Finding,S,137275,4047779,Is a
462,436096,436096,Chronic pain,Condition,SNOMED,Clinical Finding,S,436096,4118269,Is a
527,436096,436096,Chronic pain,Condition,SNOMED,Clinical Finding,S,436096,4329041,Is a
555,443783,443783,Chronic disease,Condition,SNOMED,Clinical Finding,S,443783,4274025,Is a
663,4329041,4329041,Pain,Condition,SNOMED,Clinical Finding,S,4329041,4170962,Is a


Third Level
   

In [119]:
df_agg3 = pd.DataFrame(df_concept_rel2['concept_id_2'].unique()).rename(columns={0:'SNOMED'})

In [120]:
df_concept3 = pd.merge(df_agg3, concept[['concept_id','concept_name','domain_id','vocabulary_id','concept_class_id','standard_concept']], 
                    how="left", left_on='SNOMED', right_on='concept_id')
df_concept3 = df_concept3.loc[df_concept3['standard_concept'] == 'S']


In [121]:
df_concept_rel3 = pd.merge(df_concept3,concept_relationship[['concept_id_1','concept_id_2','relationship_id']],
                     how = 'left',left_on = 'concept_id',right_on = 'concept_id_1')

df_concept_rel3 = df_concept_rel3.loc[df_concept_rel3['relationship_id'] == 'Is a']
df_concept_rel3 = df_concept_rel3.loc[df_concept_rel3['concept_name'] != 'Clinical Finding']
df_concept_rel3

,SNOMED,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_id_1,concept_id_2,relationship_id
337,4274025,4274025,Disease,Condition,SNOMED,Clinical Finding,S,4274025,441840,Is a
468,4329041,4329041,Pain,Condition,SNOMED,Clinical Finding,S,4329041,4170962,Is a
590,44783587,44783587,Finding of sensation by site,Condition,SNOMED,Clinical Finding,S,44783587,4024013,Is a
592,44783587,44783587,Finding of sensation by site,Condition,SNOMED,Clinical Finding,S,44783587,4042140,Is a
671,4024566,4024566,Muscle finding,Condition,SNOMED,Clinical Finding,S,4024566,4042140,Is a
719,4047779,4047779,Disorder by body site,Condition,SNOMED,Clinical Finding,S,4047779,4274025,Is a
773,4047779,4047779,Disorder by body site,Condition,SNOMED,Clinical Finding,S,4047779,4042140,Is a
840,4170962,4170962,Pain / sensation finding,Condition,SNOMED,Clinical Finding,S,4170962,4024013,Is a
878,4025215,4025215,Emotional state finding,Condition,SNOMED,Clinical Finding,S,4025215,4269314,Is a
1058,444363,444363,Drug-related disorder,Condition,SNOMED,Clinical Finding,S,444363,4274025,Is a


Fourth level

In [126]:
df_agg4 = pd.DataFrame(df_concept_rel3['concept_id_2'].unique()).rename(columns={0:'SNOMED'})

In [127]:
df_concept4 = pd.merge(df_agg4, concept[['concept_id','concept_name','domain_id','vocabulary_id','concept_class_id','standard_concept']], 
                    how="left", left_on='SNOMED', right_on='concept_id')
df_concept4 = df_concept4.loc[df_concept4['standard_concept'] == 'S']


In [129]:
df_concept_rel4 = pd.merge(df_concept4,concept_relationship[['concept_id_1','concept_id_2','relationship_id']],
                     how = 'left',left_on = 'concept_id',right_on = 'concept_id_1')

df_concept_rel4 = df_concept_rel4.loc[df_concept_rel4['relationship_id'] == 'Is a']
df_concept_rel4 = df_concept_rel4.loc[df_concept_rel4['concept_name'] != 'Clinical finding']
df_concept_rel4

,SNOMED,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_id_1,concept_id_2,relationship_id
392,4170962,4170962,Pain / sensation finding,Condition,SNOMED,Clinical Finding,S,4170962,4024013,Is a
403,4024013,4024013,Sensory nervous system finding,Condition,SNOMED,Clinical Finding,S,4024013,4011630,Is a
532,4042140,4042140,Finding by site,Condition,SNOMED,Clinical Finding,S,4042140,441840,Is a
899,4274025,4274025,Disease,Condition,SNOMED,Clinical Finding,S,4274025,441840,Is a
1025,4269314,4269314,Mental state finding,Condition,SNOMED,Clinical Finding,S,4269314,4293175,Is a
1190,444363,444363,Drug-related disorder,Condition,SNOMED,Clinical Finding,S,444363,4274025,Is a
1669,40483111,40483111,Mental disorder due to drug,Condition,SNOMED,Clinical Finding,S,40483111,444363,Is a
1680,40483111,40483111,Mental disorder due to drug,Condition,SNOMED,Clinical Finding,S,40483111,432586,Is a
1722,444108,444108,Finding related to sleep,Condition,SNOMED,Clinical Finding,S,444108,4041283,Is a
1767,4329041,4329041,Pain,Condition,SNOMED,Clinical Finding,S,4329041,4170962,Is a


Fifth level

In [131]:
df_agg5 = pd.DataFrame(df_concept_rel4['concept_id_2'].unique()).rename(columns={0:'SNOMED'})

In [132]:
df_concept5 = pd.merge(df_agg5, concept[['concept_id','concept_name','domain_id','vocabulary_id','concept_class_id','standard_concept']], 
                    how="left", left_on='SNOMED', right_on='concept_id')
df_concept5 = df_concept5.loc[df_concept5['standard_concept'] == 'S']


In [134]:
df_concept_rel5 = pd.merge(df_concept5,concept_relationship[['concept_id_1','concept_id_2','relationship_id']],
                     how = 'left',left_on = 'concept_id',right_on = 'concept_id_1')

df_concept_rel5 = df_concept_rel5.loc[df_concept_rel5['relationship_id'] == 'Is a']
df_concept_rel5 = df_concept_rel5.loc[df_concept_rel5['concept_name'] != 'Clinical finding']
df_concept_rel5

,SNOMED,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_id_1,concept_id_2,relationship_id
8,4024013,4024013,Sensory nervous system finding,Condition,SNOMED,Clinical Finding,S,4024013,4011630,Is a
115,4011630,4011630,Neurological finding,Condition,SNOMED,Clinical Finding,S,4011630,441840,Is a
539,4293175,4293175,"Mental state, behavior and/or psychosocial fun...",Condition,SNOMED,Clinical Finding,S,4293175,4094294,Is a
904,4274025,4274025,Disease,Condition,SNOMED,Clinical Finding,S,4274025,441840,Is a
1108,444363,444363,Drug-related disorder,Condition,SNOMED,Clinical Finding,S,444363,4274025,Is a
1605,432586,432586,Mental disorder,Condition,SNOMED,Clinical Finding,S,432586,4274025,Is a
1849,4041283,4041283,General finding of observation of patient,Condition,SNOMED,Clinical Finding,S,4041283,4094294,Is a
1991,4170962,4170962,Pain / sensation finding,Condition,SNOMED,Clinical Finding,S,4170962,4024013,Is a
2099,4042140,4042140,Finding by site,Condition,SNOMED,Clinical Finding,S,4042140,441840,Is a
2292,4180628,4180628,Disorder of body system,Condition,SNOMED,Clinical Finding,S,4180628,4047779,Is a


Sixth level

In [135]:
df_agg6 = pd.DataFrame(df_concept_rel5['concept_id_2'].unique()).rename(columns={0:'SNOMED'})

In [136]:
df_concept6 = pd.merge(df_agg6, concept[['concept_id','concept_name','domain_id','vocabulary_id','concept_class_id','standard_concept']], 
                    how="left", left_on='SNOMED', right_on='concept_id')
df_concept6 = df_concept6.loc[df_concept6['standard_concept'] == 'S']


In [137]:
df_concept_rel6 = pd.merge(df_concept6,concept_relationship[['concept_id_1','concept_id_2','relationship_id']],
                     how = 'left',left_on = 'concept_id',right_on = 'concept_id_1')

df_concept_rel6 = df_concept_rel6.loc[df_concept_rel6['relationship_id'] == 'Is a']
df_concept_rel6 = df_concept_rel6.loc[df_concept_rel6['concept_name'] != 'Clinical finding']
df_concept_rel6

,SNOMED,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_id_1,concept_id_2,relationship_id
83,4011630,4011630,Neurological finding,Condition,SNOMED,Clinical Finding,S,4011630,441840,Is a
572,4094294,4094294,Clinical history and observation findings,Condition,SNOMED,Clinical Finding,S,4094294,441840,Is a
941,4274025,4274025,Disease,Condition,SNOMED,Clinical Finding,S,4274025,441840,Is a
1041,4024013,4024013,Sensory nervous system finding,Condition,SNOMED,Clinical Finding,S,4024013,4011630,Is a
1082,4047779,4047779,Disorder by body site,Condition,SNOMED,Clinical Finding,S,4047779,4274025,Is a
1136,4047779,4047779,Disorder by body site,Condition,SNOMED,Clinical Finding,S,4047779,4042140,Is a
1300,4042140,4042140,Finding by site,Condition,SNOMED,Clinical Finding,S,4042140,441840,Is a


Seventh level

In [141]:
df_agg7 = pd.DataFrame(df_concept_rel6['concept_id_2'].unique()).rename(columns={0:'SNOMED'})

In [142]:
df_concept7 = pd.merge(df_agg7, concept[['concept_id','concept_name','domain_id','vocabulary_id','concept_class_id','standard_concept']], 
                    how="left", left_on='SNOMED', right_on='concept_id')
df_concept7 = df_concept7.loc[df_concept7['standard_concept'] == 'S']


In [143]:
df_concept_rel7 = pd.merge(df_concept7,concept_relationship[['concept_id_1','concept_id_2','relationship_id']],
                     how = 'left',left_on = 'concept_id',right_on = 'concept_id_1')

df_concept_rel7 = df_concept_rel7.loc[df_concept_rel7['relationship_id'] == 'Is a']
df_concept_rel7 = df_concept_rel7.loc[df_concept_rel7['concept_name'] != 'Clinical finding']
df_concept_rel7

,SNOMED,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_id_1,concept_id_2,relationship_id
467,4011630,4011630,Neurological finding,Condition,SNOMED,Clinical Finding,S,4011630,441840,Is a
830,4274025,4274025,Disease,Condition,SNOMED,Clinical Finding,S,4274025,441840,Is a
1027,4042140,4042140,Finding by site,Condition,SNOMED,Clinical Finding,S,4042140,441840,Is a


Eight level

In [144]:
df_agg8 = pd.DataFrame(df_concept_rel7['concept_id_2'].unique()).rename(columns={0:'SNOMED'})

In [145]:
df_concept8 = pd.merge(df_agg8, concept[['concept_id','concept_name','domain_id','vocabulary_id','concept_class_id','standard_concept']], 
                    how="left", left_on='SNOMED', right_on='concept_id')
df_concept8 = df_concept8.loc[df_concept8['standard_concept'] == 'S']


In [146]:
df_concept_rel8 = pd.merge(df_concept8,concept_relationship[['concept_id_1','concept_id_2','relationship_id']],
                     how = 'left',left_on = 'concept_id',right_on = 'concept_id_1')

df_concept_rel8 = df_concept_rel8.loc[df_concept_rel8['relationship_id'] == 'Is a']
df_concept_rel8 = df_concept_rel8.loc[df_concept_rel8['concept_name'] != 'Clinical finding']
df_concept_rel8

,SNOMED,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_id_1,concept_id_2,relationship_id


In [150]:
df_graph = df_concept_rel
df_graph = df_graph.append(df_concept_rel2)
df_graph = df_graph.append(df_concept_rel3)
df_graph = df_graph.append(df_concept_rel4)
df_graph = df_graph.append(df_concept_rel5)
df_graph = df_graph.append(df_concept_rel6)
df_graph = df_graph.append(df_concept_rel7)
df_graph = df_graph.append(df_concept_rel8)
df_graph = df_graph.drop_duplicates().reset_index()
df_graph

,index,SNOMED,Count,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_id_1,concept_id_2,relationship_id
0,2,40405599,21.0,40405599,Fibromyalgia,Condition,SNOMED,Clinical Finding,S,40405599,4160062,Is a
1,20,40405599,21.0,40405599,Fibromyalgia,Condition,SNOMED,Clinical Finding,S,40405599,4132926,Is a
2,34,40405599,21.0,40405599,Fibromyalgia,Condition,SNOMED,Clinical Finding,S,40405599,137275,Is a
3,35,40405599,21.0,40405599,Fibromyalgia,Condition,SNOMED,Clinical Finding,S,40405599,436096,Is a
4,36,40405599,21.0,40405599,Fibromyalgia,Condition,SNOMED,Clinical Finding,S,40405599,443783,Is a
...,...,...,...,...,...,...,...,...,...,...,...,...
81,2396,4199402,NaN,4199402,Finding of body region,Condition,SNOMED,Clinical Finding,S,4199402,4042140,Is a
82,2451,444112,NaN,444112,Integumentary system finding,Condition,SNOMED,Clinical Finding,S,444112,4042140,Is a
83,2492,4212577,NaN,4212577,Skin AND/OR mucosa finding,Condition,SNOMED,Clinical Finding,S,4212577,4042140,Is a
84,2606,4093991,NaN,4093991,General finding of soft tissue,Condition,SNOMED,Clinical Finding,S,4093991,4042140,Is a


In [160]:
temp = df_graph[['concept_id_1','concept_name','concept_id_2']].rename(columns ={'concept_name':'concept1_name'})
temp2 = pd.merge(temp,concept[['concept_id','concept_name']],
                     how = 'left',left_on = 'concept_id_2',right_on = 'concept_id')
df_final_graph = temp2[['concept_id_1','concept1_name','concept_id_2','concept_name']].rename(columns = {'concept_name':'concept2_name'})



In [164]:
df_final_graph.to_csv('/Users/khs2138/OneDrive - cumc.columbia.edu/Archive/Classes/Symbolic_Methods/Symbolic_Project/DC_is_a_graph.csv')

In [168]:
df_final_graph_r = df_final_graph.rename(columns = {'concept1_name':'from','concept2_name':'to'})[['from','to']]
df_final_graph_r.to_csv('/Users/khs2138/OneDrive - cumc.columbia.edu/Archive/Classes/Symbolic_Methods/Symbolic_Project/DC_is_a_graph_r.csv')
df_final_graph_r

,from,to
0,Fibromyalgia,Disorder characterized by pain
1,Fibromyalgia,Pain finding at anatomical site
2,Fibromyalgia,Disorder of muscle
3,Fibromyalgia,Chronic pain
4,Fibromyalgia,Chronic disease
...,...,...
81,Finding of body region,Finding by site
82,Integumentary system finding,Finding by site
83,Skin AND/OR mucosa finding,Finding by site
84,General finding of soft tissue,Finding by site


In [173]:
%pip install anytree

     |████████████████████████████████| 41 kB 1.9 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [174]:
from anytree import Node, RenderTree


In [175]:
def add_nodes(nodes, parent, child):
    if parent not in nodes:
        nodes[parent] = Node(parent)  
    if child not in nodes:
        nodes[child] = Node(child)
    nodes[child].parent = nodes[parent]

nodes = {}  # store references to created nodes 
# data.apply(lambda x: add_nodes(nodes, x["Parent"], x["Child"]), axis=1)  # 1-liner
for parent, child in zip(df_final_graph_r["to"],df_final_graph_r["from"]):
    add_nodes(nodes, parent, child)



In [178]:
roots = list(df_final_graph_r[~df_final_graph_r["to"].isin(df_final_graph_r["from"])]["to"].unique())
for root in roots:         # you can skip this for roots[0], if there is no forest and just 1 tree
    for pre, _, node in RenderTree(nodes[root]):
        print("%s%s" % (pre, node.name))

Finding of pattern of pain
Functional finding of respiratory tract
└── Cough
    └── Chronic cough
Clinical finding
├── Disease
│   ├── Chronic disease
│   │   └── Fibromyalgia
│   ├── Mental disorder
│   │   └── Mental disorder due to drug
│   │       └── Psychoactive substance use disorder
│   │           └── Psychoactive substance dependence
│   │               └── Alcoholism
│   ├── Drug-related disorder
│   │   ├── Disorder caused by alcohol
│   │   └── Drug dependence
│   └── Sleep disorder
│       ├── Dyssomnia
│       └── Disorders of initiating and maintaining sleep
│           └── Insomnia
├── Finding by site
│   ├── Finding of sensation by site
│   │   └── Finding of sensation of skin
│   │       └── Altered sensation of skin
│   ├── Muscle finding
│   ├── Disorder by body site
│   │   ├── Disorder of muscle
│   │   └── Disorder of body system
│   │       └── Disorder of nervous system
│   │           └── System disorder of the nervous system
│   │               └── Movement

In [ ]:
df_graph = pd.merge(df_agg, concept[['concept_id','concept_name','domain_id','vocabulary_id','concept_class_id','standard_concept']], 
                    how="left", left_on='SNOMED', right_on='concept_id')
df_graph

In [ ]:
#   parents_temp = concept_relationship.loc[(concept_relationship['concept_id_1']==SNOMED)&
 #                                               (concept_relationship['relationship_id']=='Is a') &
  #                                                  (concept_relationship['concept_id_2']!='Clinical Finding')]['concept_id_2'].values.tolist() 
   

ORIGINAL


In [ ]:
# 3b. Standard SNOMED --> All Parent/Child Relationships
def get_hierarchy(std_diag_dict):    
    snomed_hierarchy = {}
    snomed_parent = {}
    snomed_child = {}
    # Loop through all ICD codes in patient data 
    for entry in std_diag_dict:
        # Intialize a dictionary that will hold the SNOMED standard IDs and Associated MeSH concepts for each ICD code 
        sub_dict = {}
        # Add the SNOMED standard ID
        sub_dict['SNOMED'] = std_diag_dict[entry]
        # Loop through all SNOMED codes for each ICD code in patient data 
        parents = []
        children = []
        for SNOMED in std_diag_dict[entry]:
            parents_temp = concept_relationship.loc[(concept_relationship['concept_id_1']==SNOMED)&
                                                (concept_relationship['relationship_id']=='Is a')]['concept_id_2'].values.tolist()
            parents = parents + parents_temp
            children_temp = concept_relationship.loc[(concept_relationship['concept_id_1']==SNOMED)&
                                                (concept_relationship['relationship_id']=='Subsumes')]['concept_id_2'].values.tolist()
            children = children + children_temp
        sub_dict['SNOMED_parents'] = parents
        sub_dict['SNOMED_children'] = children
        snomed_hierarchy[entry] = sub_dict 
        snomed_parent[entry] = std_diag_dict[entry] + parents
        snomed_child[entry] = std_diag_dict[entry] + children
    return snomed_hierarchy,snomed_parent,snomed_child